## Import packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.2


## Load workspace and download registered model

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model=Model(ws, 'keras_reuters_best')
model.download(target_dir = '.', exist_ok = 'True')
import os 
# verify the downloaded model file
os.stat('reuters_model.h5')

Found the config file in: /data/home/erimen_cse/AMLProjects/MachineLearningNotebooks/tutorials/config.json


os.stat_result(st_mode=33204, st_ino=2097230, st_dev=2081, st_nlink=1, st_uid=1003, st_gid=1003, st_size=61755096, st_atime=1548115720, st_mtime=1548115721, st_ctime=1548115721)

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.


In [45]:
%%writefile score.py
import json
import os
import numpy as np
from keras.models import load_model
from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('keras_reuters_best')
    model = load_model(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return json.dumps(y_hat.tolist())

Overwriting score.py


### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `keras`, `tensorflow` and `azureml-sdk`.

In [9]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("keras")
#myenv.add_conda_package("tensorflow-gpu")
myenv.add_tensorflow_pip_package(core_type='cpu', version='1.11.0')
myenv.add_pip_package("azure-cli-core<2.0.55")
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [10]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
  - tensorflow==1.11.0
  - azure-cli-core<2.0.55
- keras



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [11]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Reuters",  "method" : "keras"}, 
                                               description='Predict Reuters with keras')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [47]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='keras-reuters-3',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image keras-reuters-3:1, operation "Succeeded"
Creating service
Running.......................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 1.68 s, sys: 42.4 ms, total: 1.72 s
Wall time: 7min 3s


In [48]:
print(service.get_logs())

2019-01-22T08:05:01,252265861+00:00 - gunicorn/run 
2019-01-22T08:05:01,256092913+00:00 - rsyslog/run 
ok: run: rsyslog: (pid 15) 0s
2019-01-22T08:05:01,256017410+00:00 - iot-server/run 
2019-01-22T08:05:01,257855983+00:00 - nginx/run 
ok: run: rsyslog: (pid 15) 0s
ok: run: gunicorn: (pid 13) 0s
ok: run: nginx: (pid 14) 0s
ok: run: rsyslog: (pid 15) 0s
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-01-22T08:05:01,444365578+00:00 - iot-server/finish 1 0
2019-01-22T08:05:01,445530424+00:00 - Exit code 1 is normal. Not restarting iot-server.
{"timestamp": "2019-01-22T08:05:01.692710Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-90accf8280ea4b2cad4721a169732d63-23736b9bf1d664ca708cd5", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-01-22T08:

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [49]:
print(service.scoring_uri)

http://13.83.108.51:80/score


## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model with a random sample of 30 images from the test data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 

 Since the model accuracy is high, you might have to run the following code a few times before you can see a misclassified sample.

In [16]:
import keras
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)
num_classes = max(y_train) + 1

max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [111]:
print (y_test[0:5])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [51]:
import json

# find 30 random samples from test set
# n = 30

s = 100
e = s + 30

test_samples = json.dumps({"data": x_test[s:e].tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

test_labels = y_test[s:e]
body = np.array(json.loads(test_samples)['data'])

# predict using the deployed model
result = json.loads(service.run(input_data=test_samples))

for i in range(0, len(result)):
    indice_prediction = np.argmax(result[i])
    indice_label = np.argmax(test_labels[i])
    if indice_prediction == indice_label:
        print("correct prediction for sample " + str(i))
    else:
        print("INCORRECT prediction for sample" + str(i))

INCORRECT prediction for sample0
INCORRECT prediction for sample1
correct prediction for sample 2
correct prediction for sample 3
correct prediction for sample 4
correct prediction for sample 5
correct prediction for sample 6
INCORRECT prediction for sample7
correct prediction for sample 8
INCORRECT prediction for sample9
INCORRECT prediction for sample10
correct prediction for sample 11
correct prediction for sample 12
correct prediction for sample 13
INCORRECT prediction for sample14
correct prediction for sample 15
correct prediction for sample 16
correct prediction for sample 17
correct prediction for sample 18
correct prediction for sample 19
correct prediction for sample 20
INCORRECT prediction for sample21
correct prediction for sample 22
correct prediction for sample 23
correct prediction for sample 24
correct prediction for sample 25
correct prediction for sample 26
correct prediction for sample 27
INCORRECT prediction for sample28
correct prediction for sample 29


In [112]:
import requests
import json

# send a random row from the test set to score
random_index = np.random.randint(0, len(x_test)-1)
print (random_index)
input_data = "{\"data\": [" + str(list(x_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
# print("label:", y_test[random_index])
# print("prediction:", resp.text)

import ast
result = ast.literal_eval(json.loads(resp.text))
#print (result)

indice_prediction = np.argmax(result[0])
print(indice_prediction)
indice_label = np.argmax(y_test[random_index])
print(indice_label)

1011
POST to url http://13.83.108.51:80/score
label: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
prediction: "[[7.148323493311182e-05, 0.0006692331517115235, 2.8482483685365878e-05, 0.9808841943740845, 0.0067803943529725075, 1.5119664567464497e-05, 3.5796758311335e-05, 5.306567618390545e-05, 0.0017548621399328113, 7.204800931503996e-05, 0.00029933039331808686, 0.0009929620428010821, 0.0003102721821051091, 2.1101988750160672e-05, 4.312822420615703e-05, 9.70678956946358e-06, 0.0017854924080893397, 2.1938954887446016e-05, 4.2201299947919324e-05, 0.0025818233843892813, 0.001441393862478435, 0.0004726932675112039, 2.4641873096697964e-05, 0.0001069498248398304, 1.5779825844219886e-05, 3.205460598110221e-05, 7.633689165231772e-06, 3.392386497580446e-05, 3.796141754719429e-05, 9.162038622889668e-05, 0.00013864960055798292, 5.210936797084287e-05, 2.8641175958910026e-05, 2.7729709472623654e-05, 8.613